In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import datetime
import networkx as nx


In [10]:
from operator import itemgetter

# podejście biznesowe:

2. Posiadamy dane (np. przekazane przez klienta lub pozyskane przy okazji
innych projektów) i zastanawiamy się jakie interesujące problemy badawcze
możemy przy ich pomocy rozwiązać, na jakie interesujące pytania badawcze
możemy znaleźć odpowiedź.

In [3]:
df = pd.read_csv("cleaned_bike.csv")

C:\Users\filip\AppData\Local\Temp\ipykernel_15484\1373750519.py:1: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("cleaned_bike.csv")


In [5]:
df[["departure", "return"]] = df[["departure", "return"]].apply(pd.to_datetime)

In [22]:
# create a dictionary of stations key:departure_name ,values: latitute and longitude columns
departure_stations_dict = dict(zip(df['departure_name'], zip(df['departure_latitude'], df['departure_longitude'])))
return_stations_dict = dict(zip(df['return_name'], zip(df['return_latitude'], df['return_longitude'])))


In [23]:
# merge the two dictionaries
stations_dict = {**departure_stations_dict, **return_stations_dict}

In [24]:
print(stations_dict)
print(len(stations_dict))

{'Kuusitie': (60.1952452, 24.9018997), 'Kamppi (M)': (60.1686095, 24.9305373), 'Porolahden koulu': (60.19553971683871, 25.053580944057305), 'Vallipolku': (60.227827, 24.819614), 'Länsisatamankatu': (60.1589276, 24.909692), 'Radiokatu': (60.20499, 24.918419), 'Tyynenmerenkatu': (60.156838, 24.920661), 'Itämerentori': (60.1635309, 24.9145165), 'Koskelantie': (60.208749, 24.9465139), 'Rautatieläisenkatu': (60.2014382, 24.9332721), 'Töölönkatu': (60.1775598, 24.9283272), 'Roihupelto': (60.209287, 25.055528), 'Vilhonvuorenkatu': (60.1864633, 24.9678718), 'Näkinsilta': (60.179765, 24.9554618), 'Senaatintori': (60.1691278, 24.9526414), 'Olympiastadion': (60.1845055, 24.9249741), 'Tilkanvierto': (60.1975329, 24.899261), 'Rautatientori / länsi': (60.1706061, 24.9397601), 'Pohjankulma': (60.18288, 24.794014), 'Tulisuontie': (60.20207298671284, 25.060046596074795), 'Päijänteentie': (60.1920576, 24.9642238), 'Esterinportti': (60.1975724, 24.9267808), 'Brahen puistikko': (60.1896706, 24.9481024), '

In [25]:
def bikeNetwork(df, year=2017):
    df_bike = df.copy()
    df = df[df["departure"].dt.year == year]
    df_bike.dropna(inplace=True)

    G = nx.Graph()
    G = nx.from_pandas_edgelist(
        df_bike,
        "departure_name",
        "return_name",
        edge_attr=[
            "duration",
            "distance",
            "avg_speed",
            "temperature",
        ],
    )

    for node in G.nodes:
        G.nodes[node]["coords"] = stations_dict[node]

    # due to fact that some nodes

    return G


G = bikeNetwork(df)

In [26]:
def networkStats(G):
    print("Type of G:", type(G))
    print("Number of nodes:", G.number_of_nodes())
    print("Number of edges:", G.number_of_edges())

    degrees = dict(G.degree())
    avg_degree = np.mean(list(degrees.values()))
    print("Average degree:", avg_degree)

    density = nx.density(G)
    print("Density of the graph:", density)

    closeness = nx.closeness_centrality(G)
    avg_closeness = np.mean(list(closeness.values()))
    print("Average closeness centrality:", avg_closeness)

    betweenness = nx.betweenness_centrality(G)
    avg_betweenness = np.mean(list(betweenness.values()))
    print("Average betweenness centrality:", avg_betweenness)

    triadic_closure = nx.transitivity(G)
    print("Triadic closure:", triadic_closure)

    degree_dict = dict(G.degree(G.nodes()))
    nx.set_node_attributes(G, degree_dict, "degree")

    sorted_degree = sorted(degree_dict.items(), key=itemgetter(1), reverse=True)
    print("\nTop 5 nodes by degree:\n")
    for d in sorted_degree[:5]:
        print(d)

    sorted_betweenness = sorted(betweenness.items(), key=itemgetter(1), reverse=True)

    print("\nTop 5 nodes by betweenness centrality:\n")
    for b in sorted_betweenness[:5]:
        print(b)


networkStats(G)

Type of G: <class 'networkx.classes.graph.Graph'>
Number of nodes: 347
Number of edges: 32883
Average degree: 189.52737752161383
Density of the graph: 0.5477669870566874
Average closeness centrality: 0.6936969404543839
Average betweenness centrality: 0.001341094540767463
Triadic closure: 0.7635677043925658

Top 5 nodes by degree:

('Haukilahdenkatu', 325)
('Kaisaniemenpuisto', 290)
('Paciuksenkaari', 277)
('Korjaamo', 277)
('Laajalahden aukio', 275)

Top 5 nodes by betweenness centrality:

('Haukilahdenkatu', 0.027158743962519577)
('Kaisaniemenpuisto', 0.010592802375878235)
('Mäkelänkatu', 0.005616409063680199)
('Lehtisaarentie', 0.0044663365381278)
('Itämerentori', 0.004148186838919667)
